### 0. Questions

 - What is embedding?
 - 

### 1. Import packages

First, let's import needed modules and, random seed (we'll use it if needed) and create some auxiliary functions.

In [5]:
import pandas as pd
import numpy as np
from itertools import islice
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

### 2. Data preparation

I'll be using dataset from [Spooky Author Identification](https://www.kaggle.com/c/spooky-author-identification/overview) competition

#### 2.1 Loading the data

In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

#### 2.2 Data Fields

* id - a unique identifier for each sentence
* text - some text written by one of the authors
* author - the author of the sentence (EAP: Edgar Allan Poe, HPL: HP Lovecraft; MWS: Mary Wollstonecraft Shelley)

Let's look at the data

In [4]:
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


For now, I'm going to look only at column text to look at the ways text representation can be done

#### 2.3 Data Splitting

But nevertheless let's split the data into training and validation sets.  
As soon as we have almost $20 000$ rows in `train_df` test size will be limited to $10\%$

In [4]:
train, val = train_test_split(train_df, test_size=0.1)

### 3. Text embeddings

#### 3.1 Bag of words

##### 3.1.1 One-hot vectors 

The simplest way of word representation is **one-hot vectors**. For the i-th word in the vocabulary, the vector has 1 on the i-th dimension and 0 on the rest.   
Let's do this using sklearn's `CountVectorizer` with `binary=True`

In [12]:
count_vect = CountVectorizer(binary=True)
X_train_oh = count_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_oh.shape}")

The size of the train dataset is (17621, 24151)


By default, we are not limiting the vocabulary of the model and the length of the vector for every sentence will be $24151$ - number of words in our vocab.
Although, it can be done by setting parameter `max_features` to, for example, $10000$. By doing this, vocabulary will be built considering only the top `max_features` ordered by term frequency across the corpus.

In [13]:
X_train_oh

<17621x24151 sparse matrix of type '<class 'numpy.int64'>'
	with 386753 stored elements in Compressed Sparse Row format>

It is also worth to mention, that due to the sparsity of representation (most values in word vectors will be zeros) we can save a lot of memory by only storing the non-zero parts of the feature vectors in memory. `scipy.sparse` matrices are data structures that do exactly this and they are used in `sklearn` package.

In [26]:
17621*24151

425564771

In our case only $386$ $753$ of elements out of $425$ $564$ $771$ are non-zero.

In [27]:
count_vect_lim_vocab = CountVectorizer(binary=True, max_features=10_000)
X_train_oh = count_vect_lim_vocab.fit_transform(train['text'])
X_train_oh.shape

(17621, 10000)

Now, the length of the vector is $10000$.

Let's look at the vector for the first text in our train corpus.

In [28]:
first_sentence = val['text'][205]

In [37]:
one_hot_vector = count_vect.transform([first_sentence])
one_hot_vector

<1x24151 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [38]:
one_hot_vector = one_hot_vector.toarray()
one_hot_vector

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

It is a sparse vector with $24$ $151$ elements with only $15$ elements that are not equal to zero. 
Let's find out which are they.

In [31]:
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
indices

array([  218,   764,  6746,  6857, 11669, 13280, 13483, 13969, 14600,
       15504, 19004, 20568, 20791, 21282, 23436], dtype=int64)

These are the indices of words which are present in our sentence.  
Now we are going to create index-to-word dictionary to check the result of the work of `CountVectorizer`

In [24]:
index_to_word = {index : word for word, index in count_vect.vocabulary_.items()}
dict(take(10, index_to_word.items()) )

{7891: 'extraordinary',
 13908: 'murders',
 10830: 'in',
 19605: 'snatches',
 21331: 'they',
 12287: 'learn',
 19718: 'something',
 14600: 'of',
 21286: 'the',
 23807: 'wisdom'}

In [25]:
[(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices]

[(218, 'acquired', 1),
 (764, 'an', 1),
 (6746, 'early', 1),
 (6857, 'effect', 1),
 (11669, 'it', 1),
 (13280, 'melancholy', 1),
 (13483, 'mind', 1),
 (13969, 'my', 1),
 (14600, 'of', 1),
 (15504, 'perhaps', 1),
 (19004, 'shade', 1),
 (20568, 'such', 1),
 (20791, 'surroundings', 1),
 (21282, 'that', 1),
 (23436, 'was', 1)]

Indeed, these are the indices and corresponding words from our sentence

Because we've created the `CountVectorizer` with `binary=True`. The elements are really ones and zeros.  

##### 3.1.2 One-hot vectors with counts

A little improvement over that will be using vectorizer with `binary=False`, because this way we will take counts into account.

In [32]:
count_vect = CountVectorizer(binary=False)
X_train_cv = count_vect.fit_transform(train['text'])

one_hot_vector = count_vect.transform([first_sentence]).toarray()
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in count_vect.vocabulary_.items()}

[(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices]


[(218, 'acquired', 1),
 (764, 'an', 1),
 (6746, 'early', 1),
 (6857, 'effect', 1),
 (11669, 'it', 1),
 (13280, 'melancholy', 1),
 (13483, 'mind', 1),
 (13969, 'my', 1),
 (14600, 'of', 2),
 (15504, 'perhaps', 1),
 (19004, 'shade', 1),
 (20568, 'such', 1),
 (20791, 'surroundings', 1),
 (21282, 'that', 1),
 (23436, 'was', 1)]

We can see that now the value for 'of' is 2. It doesn't only show that this article is present in the sentence, but also indicate how many times it occurs in the sentence.

In [33]:
tokenizer = count_vect.build_tokenizer()
tokenized_sentence = tokenizer(first_sentence.lower())
tokenized_sentence.count('of')

2

##### 3.1.3 N-grams

We can take into account not only words, but collocations using parameter `ngram_range` to preserve some local ordering, because by using unigrams we don't capture even that.  

In [35]:
bigram_count_vect = CountVectorizer(binary=False, ngram_range=(1,2))
X_train_bigram = bigram_count_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_bigram.shape}")
print(X_train_bigram.count_nonzero)
one_hot_vector = bigram_count_vect.transform([first_sentence]).toarray()
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in bigram_count_vect.vocabulary_.items()}

print([(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices])

The size of the train dataset is (17621, 230738)
<bound method _data_matrix.count_nonzero of <17621x230738 sparse matrix of type '<class 'numpy.int64'>'
	with 814622 stored elements in Compressed Sparse Row format>>
[(1343, 'acquired', 1), (6635, 'an', 1), (6876, 'an effect', 1), (55432, 'early', 1), (56288, 'effect', 1), (56304, 'effect of', 1), (100764, 'it', 1), (101446, 'it was', 1), (116152, 'melancholy', 1), (117782, 'mind', 1), (122534, 'my', 1), (123392, 'my mind', 1), (131315, 'of', 2), (133486, 'of melancholy', 1), (134723, 'of such', 1), (144884, 'perhaps', 1), (144889, 'perhaps an', 1), (166798, 'shade', 1), (166804, 'shade of', 1), (180390, 'such', 1), (180639, 'such surroundings', 1), (182248, 'surroundings', 1), (185419, 'that', 1), (186247, 'that my', 1), (215110, 'was', 1), (216276, 'was perhaps', 1)]


By including bigrams into calculations the size of the vector increased from $24$ $151$ to $230$ $738$, but we still can limit it using `max_features` parameter, so it is not a big deal.

##### 3.1.4 Stopwords

Removing or not removing stopwords is a controversial topic...

##### 3.1.5 Advantages and disadvantages

$"+"$:
1. Fast to train (actually, there is no training done - just counting)

$"-"$:
1. Vector dimensionality is equal to vocabulary size (`max_features`) 
2. Longer documents will have higher average count values than shorter documents, even though they might talk about the same topics
3. **One-hot vectors don't capture meaning**

#### 3.2 TF-IDF

To address second issue of bag of words approach and in attempt of trying to deal with common words (stopwords) without deleting them the next approach can be used.  

Tf means term-frequency while tf–idf means term-frequency times inverse document-frequency   

$$ \large tf-idf(t,d) = tf(i,d) * idf(t)$$  
Term frequency is $$ \large tf(i,d) = \frac{wordCount(t,d)}{length(d)}$$ 
where 
* `wordCount(t,d)` is the number of occurrences of term *t* in the document *d*
* `length(d)` is the number of words in the document *d*  

Inverse document-frequency is $$ \large idf(t, c) = \frac{size(c)}{docCount(t, c)}$$ 
where 
* `size(c)` is the number of documents in the corpora *c*
* `docCount(t, c)` is the number of documents in corpora *c* containing the term *t*  

In real life tf-idf is computed differently:
$$ \large tf(i,d) = \log{ \left(1 + \frac{wordCount(t,d)}{length(d)}\right)}$$ 
$$ \large idf(t, c) = \log{\left(1 + \frac{1 + size(c)}{1 + docCount(t, c)}\right)}$$ 

The reason to use `log` here is this. 

In [7]:
tfidf_vect = TfidfVectorizer()
X_train_tf_idf = tfidf_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_tf_idf.shape}")

The size of the train dataset is (17621, 24151)


In [8]:
X_train_tf_idf

<17621x24151 sparse matrix of type '<class 'numpy.float64'>'
	with 386753 stored elements in Compressed Sparse Row format>

In [36]:
tfidf_vect.transform([first_sentence])

<1x24151 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [39]:
index_to_word = {index : word for word, index in tfidf_vect.vocabulary_.items()}
dict(take(10, index_to_word.items()) )

{7891: 'extraordinary',
 13908: 'murders',
 10830: 'in',
 19605: 'snatches',
 21331: 'they',
 12287: 'learn',
 19718: 'something',
 14600: 'of',
 21286: 'the',
 23807: 'wisdom'}

In [44]:
tfidf_vector = tfidf_vect.transform([first_sentence])
tfidf_vector

<1x24151 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [46]:
tfidf_vector = tfidf_vector.toarray()
tfidf_vector

array([[0., 0., 0., ..., 0., 0., 0.]])

In [47]:
indices = np.where(np.any(tfidf_vector!=0, axis=0))[0]
indices

array([  218,   764,  6746,  6857, 11669, 13280, 13483, 13969, 14600,
       15504, 19004, 20568, 20791, 21282, 23436], dtype=int64)

In [48]:
[(ind, index_to_word[ind], tfidf_vector[0, ind]) for ind in indices]

[(218, 'acquired', 0.3488180510011964),
 (764, 'an', 0.1638073907669554),
 (6746, 'early', 0.3042394440717485),
 (6857, 'effect', 0.30548406420323276),
 (11669, 'it', 0.12228590729383575),
 (13280, 'melancholy', 0.32190395695457),
 (13483, 'mind', 0.236428758518773),
 (13969, 'my', 0.12350712049149236),
 (14600, 'of', 0.14554401583067025),
 (15504, 'perhaps', 0.26568396577902653),
 (19004, 'shade', 0.3704194509663842),
 (20568, 'such', 0.22053695380712085),
 (20791, 'surroundings', 0.41764271061118924),
 (21282, 'that', 0.11081145576771458),
 (23436, 'was', 0.10751569932409907)]

### TODO: 
* Finish item 3.1.4 about using stopwords 
* Add the reason to use logarithms for tf-idf
* Write down questions
    * bag of words
    * tf-idf

In [4]:
np.log2(1_000_000)

19.931568569324174

In [7]:
np.log2(2_000_000)

20.931568569324174